In [1]:
"""
初始化设置
"""
import os
import io
import re
import ast
import pdb
import clip
import json
import torch
import faiss
import chromadb
import numpy as np
from PIL import Image
from tqdm import tqdm
Image.MAX_IMAGE_PIXELS = None
import matplotlib.pyplot as plt
from transformers import AutoModel, AutoTokenizer
from transformers import CLIPProcessor, CLIPModel

### 初始化一个clip模型
clip_model, preprocess = clip.load("ViT-B/16", device="cuda")

[2024-03-07 06:39:25,370] [INFO] [real_accelerator.py:161:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [2]:
# line_mapping = {}
# file_path = "/mnt/petrelfs/liuziyu/LLM_Memory/SimplyRetrieve/classification/New_test/database/cars196/classnames.txt"
# with open(file_path, 'r') as file:
#     for line_number, line in enumerate(file):
#         line_mapping[line_number] = line.strip()  
# print(len(line_mapping))

196


In [ ]:
Birds
top_k =2 
top_number = 3
Clustering ACC: 51.632723507076285
Semantic ACC:   69.47120666503906
************************************
Pets
top_k = 3
top_number = 2
Clustering ACC: 74.12101390024529
Semantic ACC:   74.83729553222656
************************************
Flowers
top_k = 3
top_number = 1
Clustering ACC: 63.69653602211742
Semantic ACC:   53.238861083984375
************************************
Cars
top_k = 2
top_number = 1
Clustering ACC: 53.156323840318365
Semantic ACC:   63.64885711669922

************************************
Dogs
top_k = 2
top_number = 3
Clustering ACC: 50.00815850815851
Semantic ACC:   65.2044677734375
************************************

In [ ]:
from collections import Counter

### best result record above, change top_k and top_number first.
top_k = 2
top_number = 3

index_img_save_path =  "/mnt/petrelfs/liuziyu/LLM_Memory/SimplyRetrieve/classification/New_test/database/cub200/cub200_database.index"
index = faiss.read_index(index_img_save_path)

image_paths = []
gt_labels = []
val_file_path = "/mnt/petrelfs/liuziyu/LLM_Memory/SimplyRetrieve/classification/New_test/database/cub200/cub200_database.txt"
with open(val_file_path, 'r') as file:
    for line in file:
        image_path = line.split(' ')[0]
        label = line.split(' ')[1].strip()
        image_paths.append(image_path)
        gt_labels.append(label)

selected_names = []

trainset_file_path = "/mnt/petrelfs/liuziyu/LLM_Memory/SimplyRetrieve/classification/New_test/database/cub200/paired_data_birds.txt"
with open(trainset_file_path, 'r') as file:
    lines = file.readlines()
    for idx, image_path in enumerate(tqdm(image_paths, desc="Process:")):
        # print(image_path)
        # print(gt_labels[idx])
        # print(line_mapping[int(gt_labels[idx])])
    
        with torch.no_grad():
            image = preprocess(Image.open(image_path)).unsqueeze(0).to("cuda")
            # torch.Size([1, 512])
            image_features = clip_model.encode_image(image)
            image_features /= image_features.norm(dim=-1, keepdim=True)
            image_features = np.array(image_features.cpu())
            distance, index_result = index.search(image_features, top_k)

            retrieved_names = []
            for index_number in index_result[0]:
                parts = lines[index_number].strip()
                list_part = parts.split('[')[-1].rstrip(']')
                list_extracted = ast.literal_eval('[' + list_part + ']')
                retrieved_names.extend(list_extracted[:top_number])
            # print(retrieved_names)

            # most_common_element = Counter(retrieved_names).most_common(1)[0][0]
            # print(most_common_element)
            # selected_names.append(most_common_element)

            selected_names.extend(retrieved_names)
            
print(len(selected_names))
print(selected_names)